<a href="https://colab.research.google.com/github/TrisaDas0510/Retrieval-of-NO2-from-Satellite-using-RF/blob/main/NO2_retrieval_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the data
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)

    # Converting Date to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Handling missing values by filling with mean for numerical columns
    numerical_columns = ['NO2_Ground', 'Relative_Humidity', 'Solar_Radiation',
                        'Temperature', 'Wind_Speed', 'NO2_Satellite']
    for col in numerical_columns:
        df[col] = df[col].fillna(df[col].mean())

    # Dropping rows where Wind_Direction is missing
    df = df.dropna(subset=['Wind_Direction'])

    # Selecting features and target
    features = ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                'Temperature', 'Wind_Speed', 'Wind_Direction']
    X = df[features]
    y = df['NO2_Ground']

    return X, y, df

# Training the Random Forest model
def train_random_forest(X, y):
    # Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initializing and training the model
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    )
    rf_model.fit(X_train, y_train)

    # Making predictions
    y_pred = rf_model.predict(X_test)

    # Calculating performance metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return rf_model, X_train, X_test, y_train, y_test, y_pred, mse, r2

# Visualizing feature importance
def plot_feature_importance(model, features):
    importance = model.feature_importances_
    feature_importance = pd.DataFrame({'Feature': features, 'Importance': importance})
    feature_importance = feature_importance.sort_values('Importance', ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title('Feature Importance in Random Forest Model')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Actual NO2 Ground Concentration')
    plt.ylabel('Predicted NO2 Ground Concentration')
    plt.title('Actual vs Predicted NO2 Ground Concentrations')
    plt.tight_layout()
    plt.savefig('actual_vs_predicted.png')
    plt.close()

# Main execution
def main():
    # Load and preprocess data
    file_path = 'prepared_air_quality_data.csv'
    X, y, df = load_and_preprocess_data(file_path)

    # Train the model
    rf_model, X_train, X_test, y_train, y_test, y_pred, mse, r2 = train_random_forest(X, y)

    # Print performance metrics
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R-squared Score: {r2:.4f}")

    # Plot feature importance
    plot_feature_importance(rf_model, X.columns)

    # Plot actual vs predicted values
    plot_actual_vs_predicted(y_test, y_pred)

    print("Plots have been saved as 'feature_importance.png' and 'actual_vs_predicted.png'")

if __name__ == "__main__":
    main()

Mean Squared Error: 148.3497
R-squared Score: 0.5891
Plots have been saved as 'feature_importance.png' and 'actual_vs_predicted.png'


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the data
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)

    # Converting Date to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Handling duplicates by averaging
    df = df.groupby(['Date', 'Station']).mean().reset_index()

    # Handling missing values
    numerical_columns = ['NO2_Ground', 'Relative_Humidity', 'Solar_Radiation',
                        'Temperature', 'Wind_Speed', 'NO2_Satellite']
    for col in numerical_columns:
        df[col] = df[col].fillna(df[col].mean())
    df['Wind_Direction'] = df['Wind_Direction'].fillna(df['Wind_Direction'].median())

    # Removing outliers using IQR method
    def remove_outliers(df, column):
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    df = remove_outliers(df, 'NO2_Ground')

    # Feature engineering
    df['Month'] = df['Date'].dt.month
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Temp_RH_Interaction'] = df['Temperature'] * df['Relative_Humidity']
    df['Wind_Speed_Direction'] = df['Wind_Speed'] * np.cos(np.radians(df['Wind_Direction']))

    # Selecting features and target
    features = ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                'Temperature', 'Wind_Speed', 'Wind_Direction', 'Month',
                'DayOfWeek', 'Temp_RH_Interaction', 'Wind_Speed_Direction']
    X = df[features]
    y = df['NO2_Ground']

    # Scaling features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled, y, df, features, scaler

# Training the Random Forest model with hyperparameter tuning
def train_random_forest(X, y):
    # Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Defining parameter grid for GridSearchCV
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }

    # Initializing model
    rf_model = RandomForestRegressor(random_state=42)

    # Performing grid search
    grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Best model
    best_model = grid_search.best_estimator_

    # Making predictions
    y_pred = best_model.predict(X_test)

    # Calculating performance metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Cross-validation scores
    cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='r2')

    return best_model, X_train, X_test, y_train, y_test, y_pred, mse, r2, cv_scores, grid_search

# Visualizing feature importance
def plot_feature_importance(model, features):
    importance = model.feature_importances_
    feature_importance = pd.DataFrame({'Feature': features, 'Importance': importance})
    feature_importance = feature_importance.sort_values('Importance', ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title('Feature Importance in Random Forest Model')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig('feature_importance_improved.png')
    plt.close()

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Actual NO2 Ground Concentration')
    plt.ylabel('Predicted NO2 Ground Concentration')
    plt.title('Actual vs Predicted NO2 Ground Concentrations')
    plt.tight_layout()
    plt.savefig('actual_vs_predicted_improved.png')
    plt.close()

# Main execution
def main():
    # Load and preprocess data
    file_path = 'prepared_air_quality_data.csv'
    X, y, df, features, scaler = load_and_preprocess_data(file_path)

    # Train the model
    best_model, X_train, X_test, y_train, y_test, y_pred, mse, r2, cv_scores, grid_search = train_random_forest(X, y)

    # Print performance metrics
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R-squared Score: {r2:.4f}")
    print(f"Cross-validated R-squared Scores: {cv_scores}")
    print(f"Mean CV R-squared: {cv_scores.mean():.4f}")

    # Plot feature importance
    plot_feature_importance(best_model, features)

    # Plot actual vs predicted values
    plot_actual_vs_predicted(y_test, y_pred)

    print("Plots have been saved as 'feature_importance_improved.png' and 'actual_vs_predicted_improved.png'")

if __name__ == "__main__":
    main()

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 5.0049
R-squared Score: 0.5551
Cross-validated R-squared Scores: [-0.10459572  0.1261106   0.10374922 -0.09729397  0.07781373]
Mean CV R-squared: 0.0212
Plots have been saved as 'feature_importance_improved.png' and 'actual_vs_predicted_improved.png'


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the data
def load_and_preprocess_data(file_path, station):
    try:
        df = pd.read_csv(file_path)

        # Filter by station
        df = df[df['Station'] == station]

        if df.empty:
            print(f"No data available for station {station}.")
            return None, None, None, None, None

        # Converting Date to datetime
        df['Date'] = pd.to_datetime(df['Date'])

        # Select numeric columns for averaging, excluding non-numeric columns
        numeric_columns = ['NO2_Ground', 'Relative_Humidity', 'Solar_Radiation',
                          'Temperature', 'Wind_Speed', 'Wind_Direction', 'NO2_Satellite']
        df_numeric = df[['Date'] + [col for col in numeric_columns if col in df.columns]]

        # Handling duplicates by averaging numeric columns
        df_numeric = df_numeric.groupby('Date').mean().reset_index()

        # Re-merge with non-numeric columns if needed (e.g., Station for reference)
        df = df.drop_duplicates(subset=['Date'])[['Date']].merge(df_numeric, on='Date', how='left')

        # Handling missing values
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].fillna(df[col].mean())
        if 'Wind_Direction' in df.columns:
            df['Wind_Direction'] = df['Wind_Direction'].fillna(df['Wind_Direction'].median())

        # Less aggressive outlier removal (using 2 * IQR)
        def remove_outliers(df, column):
            if column in df.columns and not df[column].empty:
                Q1 = df[column].quantile(0.25)
                Q3 = df[column].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 2 * IQR
                upper_bound = Q3 + 2 * IQR
                return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
            return df

        df = remove_outliers(df, 'NO2_Ground')

        # Feature engineering
        df['Month'] = df['Date'].dt.month
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['Temp_RH_Interaction'] = df['Temperature'] * df['Relative_Humidity']
        if 'Wind_Speed' in df.columns and 'Wind_Direction' in df.columns:
            df['Wind_Speed_Direction'] = df['Wind_Speed'] * np.cos(np.radians(df['Wind_Direction']))
        else:
            df['Wind_Speed_Direction'] = 0  # Default value if Wind_Direction is missing

        # Selecting features and target
        features = ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                    'Temperature', 'Wind_Speed', 'Wind_Direction', 'Month',
                    'DayOfWeek', 'Temp_RH_Interaction', 'Wind_Speed_Direction']
        # Filter features present in df
        features = [f for f in features if f in df.columns]
        if not features or 'NO2_Ground' not in df.columns:
            print(f"Insufficient features or target for station {station}.")
            return None, None, None, None, None

        X = df[features]
        y = df['NO2_Ground']

        # Scaling features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        return X_scaled, y, df, features, scaler
    except Exception as e:
        print(f"Error preprocessing data for station {station}: {str(e)}")
        return None, None, None, None, None

# Training the Random Forest model with hyperparameter tuning
def train_random_forest(X, y):
    try:
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Defining expanded parameter grid
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 0.33]  # Removed 'auto' to avoid warning
        }

        # Initializing model
        rf_model = RandomForestRegressor(random_state=42)

        # Performing grid search with KFold
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        grid_search = GridSearchCV(rf_model, param_grid, cv=kfold, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Best model
        best_model = grid_search.best_estimator_

        # Making predictions
        y_pred = best_model.predict(X_test)

        # Calculating performance metrics
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Cross-validation scores
        cv_scores = cross_val_score(best_model, X, y, cv=kfold, scoring='r2')

        return best_model, X_train, X_test, y_train, y_test, y_pred, mse, r2, cv_scores, grid_search
    except Exception as e:
        print(f"Error training model: {str(e)}")
        return None, None, None, None, None, None, None, None, None

# Visualizing feature importance
def plot_feature_importance(model, features, station):
    try:
        importance = model.feature_importances_
        feature_importance = pd.DataFrame({'Feature': features, 'Importance': importance})
        feature_importance = feature_importance.sort_values('Importance', ascending=False)

        plt.figure(figsize=(10, 6))
        sns.barplot(x='Importance', y='Feature', data=feature_importance)
        plt.title(f'Feature Importance in Random Forest Model ({station})')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.savefig(f'feature_importance_{station}.png')
        plt.close()

        return feature_importance.head(3)
    except Exception as e:
        print(f"Error plotting feature importance for {station}: {str(e)}")
        return None

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred, station):
    try:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel('Actual NO2 Ground Concentration')
        plt.ylabel('Predicted NO2 Ground Concentration')
        plt.title(f'Actual vs Predicted NO2 Ground Concentrations ({station})')
        plt.tight_layout()
        plt.savefig(f'actual_vs_predicted_{station}.png')
        plt.close()
    except Exception as e:
        print(f"Error plotting actual vs predicted for {station}: {str(e)}")

# Main execution
def main():
    file_path = 'prepared_air_quality_data.csv'
    stations = ['Agrabad', 'Tv_Center']

    for station in stations:
        print(f"\nProcessing data for station: {station}")

        # Load and preprocess data
        X, y, df, features, scaler = load_and_preprocess_data(file_path, station)

        if X is None or y is None or len(df) < 10:
            print(f"Skipping {station} due to insufficient or invalid data.")
            continue

        # Train the model
        result = train_random_forest(X, y)
        if result[0] is None:
            print(f"Skipping {station} due to training error.")
            continue

        best_model, X_train, X_test, y_train, y_test, y_pred, mse, r2, cv_scores, grid_search = result

        # Print performance metrics
        print(f"Best Parameters: {grid_search.best_params_}")
        print(f"Mean Squared Error: {mse:.4f}")
        print(f"R-squared Score: {r2:.4f}")
        print(f"Cross-validated R-squared Scores: {cv_scores}")
        print(f"Mean CV R-squared: {cv_scores.mean():.4f}")

        # Plot feature importance and get top features
        top_features = plot_feature_importance(best_model, features, station)
        if top_features is not None:
            print(f"Top 3 Features for {station}:\n{top_features}")

        # Plot actual vs predicted values
        plot_actual_vs_predicted(y_test, y_pred, station)

        print(f"Plots saved as 'feature_importance_{station}.png' and 'actual_vs_predicted_{station}.png'")

if __name__ == "__main__":
    main()


Processing data for station: Agrabad
Best Parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 15.2211
R-squared Score: 0.6185
Cross-validated R-squared Scores: [0.62437744 0.60058049 0.64780061 0.59933524 0.71725341]
Mean CV R-squared: 0.6379
Top 3 Features for Agrabad:
             Feature  Importance
6              Month    0.307840
4         Wind_Speed    0.135628
1  Relative_Humidity    0.127209
Plots saved as 'feature_importance_Agrabad.png' and 'actual_vs_predicted_Agrabad.png'

Processing data for station: Tv_Center


/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Mean Squared Error: 0.0000
R-squared Score: 0.0000
Cross-validated R-squared Scores: [  0.         -39.11111111 -39.11111111 -39.11111111 -39.11111111]
Mean CV R-squared: -31.2889
Top 3 Features for Tv_Center:
             Feature  Importance
0      NO2_Satellite         0.0
1  Relative_Humidity         0.0
2    Solar_Radiation         0.0
Plots saved as 'feature_importance_Tv_Center.png' and 'actual_vs_predicted_Tv_Center.png'


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the data
def load_and_preprocess_data(file_path, station):
    try:
        df = pd.read_csv(file_path)

        # Filter by station
        df = df[df['Station'] == station]

        if df.empty:
            print(f"No data available for station {station}.")
            return None, None, None, None, None

        # Check for constant target
        if df['NO2_Ground'].nunique() <= 1:
            print(f"Target NO2_Ground is constant for station {station}. Skipping modeling.")
            return None, None, None, None, None

        # Converting Date to datetime
        df['Date'] = pd.to_datetime(df['Date'])

        # Select numeric columns for averaging
        numeric_columns = ['NO2_Ground', 'Relative_Humidity', 'Solar_Radiation',
                          'Temperature', 'Wind_Speed', 'Wind_Direction', 'NO2_Satellite']
        df_numeric = df[['Date'] + [col for col in numeric_columns if col in df.columns]]

        # Handling duplicates by averaging numeric columns
        df_numeric = df_numeric.groupby('Date').mean().reset_index()

        # Merge back with Date
        df = df.drop_duplicates(subset=['Date'])[['Date']].merge(df_numeric, on='Date', how='left')

        # Handling missing values
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].fillna(df[col].mean())
        if 'Wind_Direction' in df.columns:
            df['Wind_Direction'] = df['Wind_Direction'].fillna(df['Wind_Direction'].median())

        # Less aggressive outlier removal (2 * IQR)
        def remove_outliers(df, column):
            if column in df.columns and not df[column].empty:
                Q1 = df[column].quantile(0.25)
                Q3 = df[column].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 2 * IQR
                upper_bound = Q3 + 2 * IQR
                return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
            return df

        df = remove_outliers(df, 'NO2_Ground')

        # Feature engineering
        df['Month'] = df['Date'].dt.month
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['Temp_RH_Interaction'] = df['Temperature'] * df['Relative_Humidity']
        if 'Wind_Speed' in df.columns and 'Wind_Direction' in df.columns:
            df['Wind_Speed_Direction'] = df['Wind_Speed'] * np.cos(np.radians(df['Wind_Direction']))
        else:
            df['Wind_Speed_Direction'] = 0

        # Selecting features and target
        features = ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                    'Temperature', 'Wind_Speed', 'Wind_Direction', 'Month',
                    'DayOfWeek', 'Temp_RH_Interaction', 'Wind_Speed_Direction']
        features = [f for f in features if f in df.columns]
        if not features or 'NO2_Ground' not in df.columns:
            print(f"Insufficient features or target for station {station}.")
            return None, None, None, None, None

        X = df[features]
        y = df['NO2_Ground']

        # Scaling features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        return X_scaled, y, df, features, scaler
    except Exception as e:
        print(f"Error preprocessing data for station {station}: {str(e)}")
        return None, None, None, None, None

# Training and comparing Random Forest and XGBoost models
def train_models(X, y, station):
    try:
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Random Forest
        rf_param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [5, 10, 15],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2],
            'max_features': ['sqrt', 0.5]
        }
        rf_model = RandomForestRegressor(random_state=42)
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=kfold, scoring='r2', n_jobs=-1)
        rf_grid_search.fit(X_train, y_train)
        rf_best_model = rf_grid_search.best_estimator_

        # XGBoost
        xgb_param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1.0]
        }
        xgb_model = XGBRegressor(random_state=42)
        xgb_grid_search = GridSearchCV(xgb_model, xgb_param_grid, cv=kfold, scoring='r2', n_jobs=-1)
        xgb_grid_search.fit(X_train, y_train)
        xgb_best_model = xgb_grid_search.best_estimator_

        # Predictions and metrics
        models = {'Random Forest': rf_best_model, 'XGBoost': xgb_best_model}
        results = {}
        for name, model in models.items():
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            cv_scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')
            results[name] = {
                'model': model,
                'y_pred': y_pred,
                'mse': mse,
                'r2': r2,
                'cv_scores': cv_scores,
                'best_params': rf_grid_search.best_params_ if name == 'Random Forest' else xgb_grid_search.best_params_
            }

        return X_train, X_test, y_train, y_test, results
    except Exception as e:
        print(f"Error training models for {station}: {str(e)}")
        return None, None, None, None, None

# Feature selection based on importance
def select_important_features(model, features, X, threshold=0.05):
    try:
        if hasattr(model, 'feature_importances_'):
            importance = model.feature_importances_
            feature_importance = pd.DataFrame({'Feature': features, 'Importance': importance})
            selected_features = feature_importance[feature_importance['Importance'] >= threshold]['Feature'].tolist()
            if not selected_features:
                return features, X
            selected_indices = [features.index(f) for f in selected_features]
            return selected_features, X[:, selected_indices]
        return features, X
    except Exception as e:
        print(f"Error in feature selection: {str(e)}")
        return features, X

# Visualizing feature importance
def plot_feature_importance(model, features, station, model_name):
    try:
        if hasattr(model, 'feature_importances_'):
            importance = model.feature_importances_
            feature_importance = pd.DataFrame({'Feature': features, 'Importance': importance})
            feature_importance = feature_importance.sort_values('Importance', ascending=False)

            plt.figure(figsize=(10, 6))
            sns.barplot(x='Importance', y='Feature', data=feature_importance)
            plt.title(f'Feature Importance in {model_name} Model ({station})')
            plt.xlabel('Importance')
            plt.ylabel('Feature')
            plt.tight_layout()
            plt.savefig(f'feature_importance_{station}_{model_name.lower().replace(" ", "_")}.png')
            plt.close()

            return feature_importance.head(3)
        return None
    except Exception as e:
        print(f"Error plotting feature importance for {station} ({model_name}): {str(e)}")
        return None

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred, station, model_name):
    try:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel('Actual NO2 Ground Concentration')
        plt.ylabel('Predicted NO2 Ground Concentration')
        plt.title(f'Actual vs Predicted NO2 Ground Concentrations ({station}, {model_name})')
        plt.tight_layout()
        plt.savefig(f'actual_vs_predicted_{station}_{model_name.lower().replace(" ", "_")}.png')
        plt.close()
    except Exception as e:
        print(f"Error plotting actual vs predicted for {station} ({model_name}): {str(e)}")

# Main execution
def main():
    file_path = 'prepared_air_quality_data.csv'
    stations = ['Agrabad']  # Excluding Tv_Center due to constant data

    for station in stations:
        print(f"\nProcessing data for station: {station}")

        # Load and preprocess data
        X, y, df, features, scaler = load_and_preprocess_data(file_path, station)

        if X is None or y is None or len(df) < 10:
            print(f"Skipping {station} due to insufficient or invalid data.")
            continue

        # Train models
        result = train_models(X, y, station)
        if result[0] is None:
            print(f"Skipping {station} due to training error.")
            continue

        X_train, X_test, y_train, y_test, results = result

        # Process each model
        for model_name, res in results.items():
            print(f"\nResults for {model_name} ({station}):")
            print(f"Best Parameters: {res['best_params']}")
            print(f"Mean Squared Error: {res['mse']:.4f}")
            print(f"R-squared Score: {res['r2']:.4f}")
            print(f"Cross-validated R-squared Scores: {res['cv_scores']}")
            print(f"Mean CV R-squared: {res['cv_scores'].mean():.4f}")

            # Feature selection
            selected_features, X_selected = select_important_features(res['model'], features, X, threshold=0.05)
            print(f"Selected Features: {selected_features}")

            # Plot feature importance
            top_features = plot_feature_importance(res['model'], selected_features, station, model_name)
            if top_features is not None:
                print(f"Top 3 Features:\n{top_features}")

            # Plot actual vs predicted
            plot_actual_vs_predicted(y_test, res['y_pred'], station, model_name)

            print(f"Plots saved as 'feature_importance_{station}_{model_name.lower().replace(' ', '_')}.png' and 'actual_vs_predicted_{station}_{model_name.lower().replace(' ', '_')}.png'")

if __name__ == "__main__":
    main()


Processing data for station: Agrabad

Results for Random Forest (Agrabad):
Best Parameters: {'max_depth': 10, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 15.3311
R-squared Score: 0.6157
Cross-validated R-squared Scores: [0.61350302 0.62270001 0.6544858  0.6375751  0.7370683 ]
Mean CV R-squared: 0.6531
Selected Features: ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Wind_Speed', 'Month', 'Temp_RH_Interaction']
Error plotting feature importance for Agrabad (Random Forest): All arrays must be of the same length
Plots saved as 'feature_importance_Agrabad_random_forest.png' and 'actual_vs_predicted_Agrabad_random_forest.png'

Results for XGBoost (Agrabad):
Best Parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.8}
Mean Squared Error: 16.5551
R-squared Score: 0.5851
Cross-validated R-squared Scores: [0.61844366 0.50005039 0.66234864 0.60173751 0.70945308]
Mean CV R-squared: 0.61

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the data
def load_and_preprocess_data(file_path, station, min_samples=10):
    try:
        df = pd.read_csv(file_path)

        # Filter by station
        df = df[df['Station'] == station]

        if df.empty:
            print(f"No data available for station {station}.")
            return None, None, None, None, None

        # Check for constant or insufficient target data
        if df['NO2_Ground'].nunique() <= 1:
            print(f"Target NO2_Ground is constant for station {station}. Skipping modeling.")
            return None, None, None, None, None
        if len(df) < min_samples:
            print(f"Insufficient samples ({len(df)}) for station {station}. Minimum required: {min_samples}.")
            return None, None, None, None, None

        # Converting Date to datetime
        df['Date'] = pd.to_datetime(df['Date'])

        # Select numeric columns for averaging
        numeric_columns = ['NO2_Ground', 'Relative_Humidity', 'Solar_Radiation',
                          'Temperature', 'Wind_Speed', 'Wind_Direction', 'NO2_Satellite']
        df_numeric = df[['Date'] + [col for col in numeric_columns if col in df.columns]]

        # Handling duplicates by averaging numeric columns
        df_numeric = df_numeric.groupby('Date').mean().reset_index()

        # Merge back with Date
        df = df.drop_duplicates(subset=['Date'])[['Date']].merge(df_numeric, on='Date', how='left')

        # Handling missing values
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].fillna(df[col].mean())
        if 'Wind_Direction' in df.columns:
            df['Wind_Direction'] = df['Wind_Direction'].fillna(df['Wind_Direction'].median())

        # Check for constant features and remove them
        for col in df.columns:
            if col not in ['Date'] and df[col].nunique() <= 1:
                print(f"Feature {col} is constant for station {station}. Dropping.")
                df = df.drop(columns=[col])

        # Less aggressive outlier removal (2 * IQR)
        def remove_outliers(df, column):
            if column in df.columns and not df[column].empty:
                Q1 = df[column].quantile(0.25)
                Q3 = df[column].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 2 * IQR
                upper_bound = Q3 + 2 * IQR
                return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
            return df

        df = remove_outliers(df, 'NO2_Ground')

        # Feature engineering
        df['Month'] = df['Date'].dt.month
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        if 'Temperature' in df.columns and 'Relative_Humidity' in df.columns:
            df['Temp_RH_Interaction'] = df['Temperature'] * df['Relative_Humidity']
        else:
            df['Temp_RH_Interaction'] = 0
        if 'Wind_Speed' in df.columns and 'Wind_Direction' in df.columns:
            df['Wind_Speed_Direction'] = df['Wind_Speed'] * np.cos(np.radians(df['Wind_Direction']))
        else:
            df['Wind_Speed_Direction'] = 0

        # Selecting features and target
        features = ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                    'Temperature', 'Wind_Speed', 'Wind_Direction', 'Month',
                    'DayOfWeek', 'Temp_RH_Interaction', 'Wind_Speed_Direction']
        features = [f for f in features if f in df.columns]
        if not features or 'NO2_Ground' not in df.columns:
            print(f"Insufficient features or target for station {station}.")
            return None, None, None, None, None

        X = df[features]
        y = df['NO2_Ground']

        # Scaling features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        return X_scaled, y, df, features, scaler
    except Exception as e:
        print(f"Error preprocessing data for station {station}: {str(e)}")
        return None, None, None, None, None

# Training and comparing Random Forest and XGBoost models
def train_models(X, y, station):
    try:
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Random Forest
        rf_param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [5, 10, 15],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2],
            'max_features': ['sqrt', 0.5]
        }
        rf_model = RandomForestRegressor(random_state=42)
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=kfold, scoring='r2', n_jobs=-1)
        rf_grid_search.fit(X_train, y_train)
        rf_best_model = rf_grid_search.best_estimator_

        # XGBoost
        xgb_param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1.0]
        }
        xgb_model = XGBRegressor(random_state=42)
        xgb_grid_search = GridSearchCV(xgb_model, xgb_param_grid, cv=kfold, scoring='r2', n_jobs=-1)
        xgb_grid_search.fit(X_train, y_train)
        xgb_best_model = xgb_grid_search.best_estimator_

        # Predictions and metrics
        models = {'Random Forest': rf_best_model, 'XGBoost': xgb_best_model}
        results = {}
        for name, model in models.items():
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            cv_scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')
            results[name] = {
                'model': model,
                'y_pred': y_pred,
                'mse': mse,
                'r2': r2,
                'cv_scores': cv_scores,
                'best_params': rf_grid_search.best_params_ if name == 'Random Forest' else xgb_grid_search.best_params_
            }

        return X_train, X_test, y_train, y_test, results
    except Exception as e:
        print(f"Error training models for {station}: {str(e)}")
        return None, None, None, None, None

# Feature selection based on importance
def select_important_features(model, features, X, threshold=0.05):
    try:
        if hasattr(model, 'feature_importances_'):
            importance = model.feature_importances_
            feature_importance = pd.DataFrame({'Feature': features, 'Importance': importance})
            selected_features = feature_importance[feature_importance['Importance'] >= threshold]['Feature'].tolist()
            if not selected_features:
                return features, X
            selected_indices = [features.index(f) for f in selected_features]
            return selected_features, X[:, selected_indices]
        return features, X
    except Exception as e:
        print(f"Error in feature selection: {str(e)}")
        return features, X

# Visualizing feature importance
def plot_feature_importance(model, features, station, model_name):
    try:
        if hasattr(model, 'feature_importances_'):
            importance = model.feature_importances_
            feature_importance = pd.DataFrame({'Feature': features, 'Importance': importance})
            feature_importance = feature_importance.sort_values('Importance', ascending=False)

            plt.figure(figsize=(10, 6))
            sns.barplot(x='Importance', y='Feature', data=feature_importance)
            plt.title(f'Feature Importance in {model_name} Model ({station})')
            plt.xlabel('Importance')
            plt.ylabel('Feature')
            plt.tight_layout()
            plt.savefig(f'feature_importance_{station}_{model_name.lower().replace(" ", "_")}.png')
            plt.close()

            return feature_importance.head(3)
        return None
    except Exception as e:
        print(f"Error plotting feature importance for {station} ({model_name}): {str(e)}")
        return None

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred, station, model_name):
    try:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel('Actual NO2 Ground Concentration')
        plt.ylabel('Predicted NO2 Ground Concentration')
        plt.title(f'Actual vs Predicted NO2 Ground Concentrations ({station}, {model_name})')
        plt.tight_layout()
        plt.savefig(f'actual_vs_predicted_{station}_{model_name.lower().replace(" ", "_")}.png')
        plt.close()
    except Exception as e:
        print(f"Error plotting actual vs predicted for {station} ({model_name}): {str(e)}")

# Main execution
def main():
    file_path = 'prepared_air_quality_data.csv'

    # Load dataset to get all stations
    try:
        df = pd.read_csv(file_path)
        stations = df['Station'].unique()
        print(f"Found stations: {list(stations)}")
    except Exception as e:
        print(f"Error loading dataset: {str(e)}")
        return

    for station in stations:
        print(f"\nProcessing data for station: {station}")

        # Load and preprocess data
        X, y, df, features, scaler = load_and_preprocess_data(file_path, station)

        if X is None or y is None or len(df) < 10:
            print(f"Skipping {station} due to insufficient or invalid data.")
            continue

        # Train models
        result = train_models(X, y, station)
        if result[0] is None:
            print(f"Skipping {station} due to training error.")
            continue

        X_train, X_test, y_train, y_test, results = result

        # Process each model
        for model_name, res in results.items():
            print(f"\nResults for {model_name} ({station}):")
            print(f"Best Parameters: {res['best_params']}")
            print(f"Mean Squared Error: {res['mse']:.4f}")
            print(f"R-squared Score: {res['r2']:.4f}")
            print(f"Cross-validated R-squared Scores: {res['cv_scores']}")
            print(f"Mean CV R-squared: {res['cv_scores'].mean():.4f}")

            # Feature selection
            selected_features, X_selected = select_important_features(res['model'], features, X, threshold=0.05)
            print(f"Selected Features: {selected_features}")

            # Plot feature importance
            top_features = plot_feature_importance(res['model'], selected_features, station, model_name)
            if top_features is not None:
                print(f"Top 3 Features:\n{top_features}")

            # Plot actual vs predicted
            plot_actual_vs_predicted(y_test, res['y_pred'], station, model_name)

            print(f"Plots saved as 'feature_importance_{station}_{model_name.lower().replace(' ', '_')}.png' and 'actual_vs_predicted_{station}_{model_name.lower().replace(' ', '_')}.png'")

if __name__ == "__main__":
    main()

Found stations: ['Agrabad', 'BRAC', 'Barisal', 'Cumilla', 'Darussalam', 'DoE', 'Gazipur', 'Khulna', 'Mymensingh', 'NG', 'NS', 'Rajshahi', 'Rangpur', 'Savar', 'Sylhet', 'TV_Center', 'Tv_Center']

Processing data for station: Agrabad

Results for Random Forest (Agrabad):
Best Parameters: {'max_depth': 10, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 15.3311
R-squared Score: 0.6157
Cross-validated R-squared Scores: [0.61350302 0.62270001 0.6544858  0.6375751  0.7370683 ]
Mean CV R-squared: 0.6531
Selected Features: ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Wind_Speed', 'Month', 'Temp_RH_Interaction']
Error plotting feature importance for Agrabad (Random Forest): All arrays must be of the same length
Plots saved as 'feature_importance_Agrabad_random_forest.png' and 'actual_vs_predicted_Agrabad_random_forest.png'

Results for XGBoost (Agrabad):
Best Parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimato

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the combined data
def load_and_preprocess_data(file_path, exclude_stations=['tv_center'], min_samples=10):
    try:
        df = pd.read_csv(file_path)

        # Standardize station names to lowercase
        df['Station'] = df['Station'].str.lower()
        exclude_stations = [s.lower() for s in exclude_stations]

        # Exclude specified stations
        df = df[~df['Station'].isin(exclude_stations)]

        if df.empty:
            print(f"No data available after excluding stations {exclude_stations}.")
            return None, None, None, None, None, None

        # Check for constant or insufficient target data
        if df['NO2_Ground'].nunique() <= 1:
            print(f"Target NO2_Ground is constant in combined data. Skipping modeling.")
            return None, None, None, None, None, None
        if len(df) < min_samples:
            print(f"Insufficient samples ({len(df)}) in combined data. Minimum required: {min_samples}.")
            return None, None, None, None, None, None

        # Converting Date to datetime
        df['Date'] = pd.to_datetime(df['Date'])

        # Select numeric columns for averaging
        numeric_columns = ['NO2_Ground', 'Relative_Humidity', 'Solar_Radiation',
                          'Temperature', 'Wind_Speed', 'Wind_Direction', 'NO2_Satellite']
        df_numeric = df[['Date', 'Station'] + [col for col in numeric_columns if col in df.columns]]

        # Handling duplicates by averaging numeric columns per Date and Station
        df_numeric = df_numeric.groupby(['Date', 'Station']).mean().reset_index()

        # Handling missing values
        for col in numeric_columns:
            if col in df_numeric.columns:
                df_numeric[col] = df_numeric[col].fillna(df_numeric[col].mean())
        if 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Direction'] = df_numeric['Wind_Direction'].fillna(df_numeric['Wind_Direction'].median())

        # Scale NO2_Ground
        target_scaler = StandardScaler()
        df_numeric['NO2_Ground'] = target_scaler.fit_transform(df_numeric[['NO2_Ground']])

        # Check for constant features and remove them
        for col in df_numeric.columns:
            if col not in ['Date', 'Station', 'NO2_Ground'] and df_numeric[col].nunique() <= 1:
                print(f"Feature {col} is constant in combined data. Dropping.")
                df_numeric = df_numeric.drop(columns=[col])

        # Stricter outlier removal (1.5 * IQR)
        def remove_outliers(df, column):
            if column in df.columns and not df[column].empty:
                Q1 = df[column].quantile(0.25)
                Q3 = df[column].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
            return df

        df_numeric = remove_outliers(df_numeric, 'NO2_Ground')

        # Feature engineering
        df_numeric['Month'] = df_numeric['Date'].dt.month
        df_numeric['DayOfWeek'] = df_numeric['Date'].dt.dayofweek
        if 'Temperature' in df_numeric.columns and 'Relative_Humidity' in df_numeric.columns:
            df_numeric['Temp_RH_Interaction'] = df_numeric['Temperature'] * df_numeric['Relative_Humidity']
        else:
            df_numeric['Temp_RH_Interaction'] = 0
        if 'Wind_Speed' in df_numeric.columns and 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Speed_Direction'] = df_numeric['Wind_Speed'] * np.cos(np.radians(df_numeric['Wind_Direction']))
        else:
            df_numeric['Wind_Speed_Direction'] = 0

        # Selecting features and target
        features = ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                    'Temperature', 'Wind_Speed', 'Wind_Direction', 'Month',
                    'DayOfWeek', 'Temp_RH_Interaction', 'Wind_Speed_Direction', 'Station']
        features = [f for f in features if f in df_numeric.columns]
        if not features or 'NO2_Ground' not in df_numeric.columns:
            print(f"Insufficient features or target in combined data.")
            return None, None, None, None, None, None

        # Correlation analysis
        if 'NO2_Satellite' in df_numeric.columns:
            correlation = df_numeric[['NO2_Satellite', 'NO2_Ground']].corr().iloc[0, 1]
            print(f"Correlation between NO2_Satellite and NO2_Ground in combined data: {correlation:.4f}")

        X = df_numeric[features]
        y = df_numeric['NO2_Ground']

        # Define preprocessing pipeline
        numeric_features = [f for f in features if f != 'Station']
        categorical_features = ['Station'] if 'Station' in features else []

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_features),
                ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
            ])

        return X, y, df_numeric, features, preprocessor, target_scaler
    except Exception as e:
        print(f"Error preprocessing combined data: {str(e)}")
        return None, None, None, None, None, None

# Training Random Forest model
def train_random_forest(X, y, preprocessor):
    try:
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Define pipeline
        rf_param_grid = {
            'rf__n_estimators': [100, 200],
            'rf__max_depth': [5, 10, 15],
            'rf__min_samples_split': [2, 5],
            'rf__min_samples_leaf': [1, 2],
            'rf__max_features': ['sqrt', 0.5]
        }
        rf_model = RandomForestRegressor(random_state=42)
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('rf', rf_model)
        ])

        # Grid search with KFold
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        grid_search = GridSearchCV(pipeline, rf_param_grid, cv=kfold, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Best model
        best_model = grid_search.best_estimator_

        # Predictions
        y_pred = best_model.predict(X_test)

        # Metrics
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        cv_scores = cross_val_score(best_model, X, y, cv=kfold, scoring='r2')

        return best_model, X_train, X_test, y_train, y_test, y_pred, mse, r2, cv_scores, grid_search
    except Exception as e:
        print(f"Error training Random Forest model: {str(e)}")
        return None, None, None, None, None, None, None, None, None, None

# Visualizing feature importance
def plot_feature_importance(model, features, numeric_features, categorical_features):
    try:
        # Extract feature importances
        importance = model.named_steps['rf'].feature_importances_
        # Get feature names after preprocessing
        preprocessor = model.named_steps['preprocessor']
        num_features = numeric_features
        if categorical_features:
            cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        else:
            cat_feature_names = []
        all_features = num_features + cat_feature_names

        feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importance})
        feature_importance = feature_importance.sort_values('Importance', ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance)
        plt.title('Feature Importance in Random Forest Model (Combined Stations)')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.savefig('feature_importance_combined_stations.png')
        plt.close()

        return feature_importance.head(3)
    except Exception as e:
        print(f"Error plotting feature importance: {str(e)}")
        return None

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred):
    try:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel('Actual NO2 Ground Concentration (Scaled)')
        plt.ylabel('Predicted NO2 Ground Concentration (Scaled)')
        plt.title('Actual vs Predicted NO2 Ground Concentrations (Combined Stations)')
        plt.tight_layout()
        plt.savefig('actual_vs_predicted_combined_stations.png')
        plt.close()
    except Exception as e:
        print(f"Error plotting actual vs predicted: {str(e)}")

# Main execution
def main():
    file_path = 'prepared_air_quality_data.csv'

    # Load and preprocess data
    print("Loading and preprocessing data...")
    result = load_and_preprocess_data(file_path, exclude_stations=['TV_Center', 'Tv_Center'])
    if result[0] is None:
        print("Failed to preprocess data. Exiting.")
        return

    X, y, df, features, preprocessor, target_scaler = result
    print(f"Combined data from stations: {df['Station'].unique().tolist()}")

    # Train Random Forest
    print("Training Random Forest model...")
    result = train_random_forest(X, y, preprocessor)
    if result[0] is None:
        print("Failed to train model. Exiting.")
        return

    best_model, X_train, X_test, y_train, y_test, y_pred, mse, r2, cv_scores, grid_search = result

    # Print results
    print("\nResults for Random Forest (Combined Stations):")
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R-squared Score: {r2:.4f}")
    print(f"Cross-validated R-squared Scores: {cv_scores}")
    print(f"Mean CV R-squared: {cv_scores.mean():.4f}")

    # Plot feature importance
    numeric_features = [f for f in features if f != 'Station']
    categorical_features = ['Station'] if 'Station' in features else []
    top_features = plot_feature_importance(best_model, features, numeric_features, categorical_features)
    if top_features is not None:
        print(f"Top 3 Features:\n{top_features}")

    # Plot actual vs predicted
    plot_actual_vs_predicted(y_test, y_pred)

    print("Plots saved as 'feature_importance_combined_stations.png' and 'actual_vs_predicted_combined_stations.png'")

if __name__ == "__main__":
    main()

Loading and preprocessing data...
Correlation between NO2_Satellite and NO2_Ground in combined data: -0.0063
Combined data from stations: ['agrabad', 'barisal', 'cumilla', 'darussalam', 'doe', 'gazipur', 'khulna', 'mymensingh', 'ng', 'ns', 'rangpur', 'savar', 'sylhet', 'rajshahi', 'brac']
Training Random Forest model...

Results for Random Forest (Combined Stations):
Best Parameters: {'rf__max_depth': 15, 'rf__max_features': 0.5, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 200}
Mean Squared Error: 0.0080
R-squared Score: 0.6292
Cross-validated R-squared Scores: [0.63099482 0.62084351 0.60657378 0.62064993 0.63040297]
Mean CV R-squared: 0.6219
Top 3 Features:
            Feature  Importance
10  Station_agrabad    0.127563
2   Solar_Radiation    0.112948
6             Month    0.106490
Plots saved as 'feature_importance_combined_stations.png' and 'actual_vs_predicted_combined_stations.png'


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the combined data
def load_and_preprocess_data(file_path, exclude_stations=['tv_center'], min_samples=10):
    try:
        df = pd.read_csv(file_path)

        # Standardize station names to lowercase
        df['Station'] = df['Station'].str.lower()
        exclude_stations = [s.lower() for s in exclude_stations]

        # Exclude specified stations
        df = df[~df['Station'].isin(exclude_stations)]

        if df.empty:
            print(f"No data available after excluding stations {exclude_stations}.")
            return None, None, None, None, None, None

        # Check for constant or insufficient target data
        if df['NO2_Ground'].nunique() <= 1:
            print(f"Target NO2_Ground is constant in combined data. Skipping modeling.")
            return None, None, None, None, None, None
        if len(df) < min_samples:
            print(f"Insufficient samples ({len(df)}) in combined data. Minimum required: {min_samples}.")
            return None, None, None, None, None, None

        # Converting Date to datetime
        df['Date'] = pd.to_datetime(df['Date'])

        # Select numeric columns for averaging
        numeric_columns = ['NO2_Ground', 'NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                          'Temperature', 'Wind_Speed', 'Wind_Direction']
        df_numeric = df[['Date', 'Station'] + [col for col in numeric_columns if col in df.columns]]

        # Handling duplicates by averaging numeric columns per Date and Station
        df_numeric = df_numeric.groupby(['Date', 'Station']).mean().reset_index()

        # Handling missing values
        for col in numeric_columns:
            if col in df_numeric.columns:
                df_numeric[col] = df_numeric[col].fillna(df_numeric[col].mean())
        if 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Direction'] = df_numeric['Wind_Direction'].fillna(df_numeric['Wind_Direction'].median())

        # Log-transform skewed features
        for col in ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Wind_Speed']:
            if col in df_numeric.columns:
                df_numeric[col] = np.log1p(df_numeric[col].clip(lower=0))  # Avoid log(0)

        # Scale NO2_Ground
        target_scaler = StandardScaler()
        df_numeric['NO2_Ground'] = target_scaler.fit_transform(df_numeric[['NO2_Ground']])

        # Check for constant features and remove them
        for col in df_numeric.columns:
            if col not in ['Date', 'Station', 'NO2_Ground'] and df_numeric[col].nunique() <= 1:
                print(f"Feature {col} is constant in combined data. Dropping.")
                df_numeric = df_numeric.drop(columns=[col])

        # Stricter outlier removal (1.5 * IQR)
        def remove_outliers(df, column):
            if column in df.columns and not df[column].empty:
                Q1 = df[column].quantile(0.25)
                Q3 = df[column].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
            return df

        df_numeric = remove_outliers(df_numeric, 'NO2_Ground')

        # Feature engineering
        df_numeric['Month'] = df_numeric['Date'].dt.month
        df_numeric['DayOfWeek'] = df_numeric['Date'].dt.dayofweek
        if 'Temperature' in df_numeric.columns and 'Relative_Humidity' in df_numeric.columns:
            df_numeric['Temp_RH_Interaction'] = df_numeric['Temperature'] * df_numeric['Relative_Humidity']
        else:
            df_numeric['Temp_RH_Interaction'] = 0
        if 'Wind_Speed' in df_numeric.columns and 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Speed_Direction'] = df_numeric['Wind_Speed'] * np.cos(np.radians(df_numeric['Wind_Direction']))
        else:
            df_numeric['Wind_Speed_Direction'] = 0
        if 'NO2_Satellite' in df_numeric.columns and 'Solar_Radiation' in df_numeric.columns:
            df_numeric['Satellite_Solar_Interaction'] = df_numeric['NO2_Satellite'] * df_numeric['Solar_Radiation']
        else:
            df_numeric['Satellite_Solar_Interaction'] = 0
        if 'NO2_Satellite' in df_numeric.columns:
            df_numeric['Satellite_Month_Interaction'] = df_numeric['NO2_Satellite'] * df_numeric['Month']
        else:
            df_numeric['Satellite_Month_Interaction'] = 0

        # Station clustering
        cluster_features = ['NO2_Ground', 'NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Temperature']
        cluster_data = df_numeric[[f for f in cluster_features if f in df_numeric.columns]].fillna(0)
        if len(cluster_data) > 0:
            kmeans = KMeans(n_clusters=3, random_state=42)
            df_numeric['Cluster'] = kmeans.fit_predict(cluster_data)
        else:
            df_numeric['Cluster'] = 0

        # Selecting features and target
        features = ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Temperature',
                    'Wind_Speed', 'Wind_Direction', 'Month', 'DayOfWeek', 'Temp_RH_Interaction',
                    'Wind_Speed_Direction', 'Satellite_Solar_Interaction', 'Satellite_Month_Interaction',
                    'Station', 'Cluster']
        features = [f for f in features if f in df_numeric.columns]
        if not features or 'NO2_Ground' not in df_numeric.columns:
            print(f"Insufficient features or target in combined data.")
            return None, None, None, None, None, None

        # Correlation analysis
        if 'NO2_Satellite' in df_numeric.columns:
            correlation = df_numeric[['NO2_Satellite', 'NO2_Ground']].corr().iloc[0, 1]
            print(f"Correlation between NO2_Satellite and NO2_Ground in combined data (after log-transform): {correlation:.4f}")

        X = df_numeric[features]
        y = df_numeric['NO2_Ground']

        # Define preprocessing pipeline
        numeric_features = [f for f in features if f not in ['Station', 'Cluster']]
        categorical_features = [f for f in ['Station', 'Cluster'] if f in features]

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_features),
                ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
            ])

        return X, y, df_numeric, features, preprocessor, target_scaler
    except Exception as e:
        print(f"Error preprocessing combined data: {str(e)}")
        return None, None, None, None, None, None

# Training and comparing models
def train_models(X, y, preprocessor, df):
    try:
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Get station labels for test set
        test_indices = X_test.index
        stations_test = df.loc[test_indices, 'Station']

        # Define models and pipelines
        models = {
            'Random Forest': Pipeline([
                ('preprocessor', preprocessor),
                ('rf', RandomForestRegressor(random_state=42))
            ]),
            'XGBoost': Pipeline([
                ('preprocessor', preprocessor),
                ('xgb', XGBRegressor(random_state=42))
            ]),
            'Linear Regression': Pipeline([
                ('preprocessor', preprocessor),
                ('lr', LinearRegression())
            ])
        }

        # Parameter grids
        rf_param_grid = {
            'rf__n_estimators': [100, 200],
            'rf__max_depth': [5, 10],
            'rf__min_samples_split': [2, 5],
            'rf__min_samples_leaf': [1, 2],
            'rf__max_features': ['sqrt', 0.5]
        }
        xgb_param_grid = {
            'xgb__n_estimators': [100, 200],
            'xgb__max_depth': [3, 5],
            'xgb__learning_rate': [0.01, 0.1],
            'xgb__subsample': [0.8, 1.0]
        }

        # Grid search for Random Forest and XGBoost
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        results = {}
        for name, pipeline in models.items():
            if name in ['Random Forest', 'XGBoost']:
                param_grid = rf_param_grid if name == 'Random Forest' else xgb_param_grid
                grid_search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1)
                grid_search.fit(X_train, y_train)
                best_model = grid_search.best_estimator_
                best_params = grid_search.best_params_
            else:  # Linear Regression
                pipeline.fit(X_train, y_train)
                best_model = pipeline
                best_params = None

            # Predictions and metrics
            y_pred = best_model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            cv_scores = cross_val_score(best_model, X, y, cv=kfold, scoring='r2')

            # Per-station performance
            station_metrics = {}
            for station in stations_test.unique():
                mask = stations_test == station
                if mask.sum() > 0:
                    station_mse = mean_squared_error(y_test[mask], y_pred[mask])
                    station_r2 = r2_score(y_test[mask], y_pred[mask])
                    station_metrics[station] = {'MSE': station_mse, 'R2': station_r2}

            results[name] = {
                'model': best_model,
                'y_pred': y_pred,
                'mse': mse,
                'r2': r2,
                'cv_scores': cv_scores,
                'best_params': best_params,
                'station_metrics': station_metrics
            }

        return X_train, X_test, y_train, y_test, results
    except Exception as e:
        print(f"Error training models: {str(e)}")
        return None, None, None, None, None

# Feature selection based on importance
def select_important_features(model, name, features, numeric_features, categorical_features, X, threshold=0.1):
    try:
        if name == 'Linear Regression':
            importance = np.abs(model.named_steps['lr'].coef_)
        else:
            importance = model.named_steps[name.lower().replace(' ', '_')].feature_importances_

        # Get feature names after preprocessing
        preprocessor = model.named_steps['preprocessor']
        num_features = numeric_features
        if categorical_features:
            cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        else:
            cat_feature_names = []
        all_features = num_features + cat_feature_names

        feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importance})
        selected_features = feature_importance[feature_importance['Importance'] >= threshold]['Feature'].tolist()

        if not selected_features:
            return num_features + cat_feature_names, X

        # Map back to original feature names
        selected_orig_features = []
        for f in selected_features:
            if f in num_features:
                selected_orig_features.append(f)
            else:
                # Handle categorical features
                for cat_feature in categorical_features:
                    if f.startswith(cat_feature):
                        if cat_feature not in selected_orig_features:
                            selected_orig_features.append(cat_feature)

        # Select corresponding columns in X
        selected_cols = []
        for f in selected_orig_features:
            if f in numeric_features or f in categorical_features:
                selected_cols.append(f)

        return selected_cols, X[selected_cols]
    except Exception as e:
        print(f"Error in feature selection for {name}: {str(e)}")
        return features, X

# Visualizing feature importance
def plot_feature_importance(model, name, features, numeric_features, categorical_features):
    try:
        if name == 'Linear Regression':
            importance = np.abs(model.named_steps['lr'].coef_)
        else:
            importance = model.named_steps[name.lower().replace(' ', '_')].feature_importances_

        # Get feature names after preprocessing
        preprocessor = model.named_steps['preprocessor']
        num_features = numeric_features
        if categorical_features:
            cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        else:
            cat_feature_names = []
        all_features = num_features + cat_feature_names

        feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importance})
        feature_importance = feature_importance.sort_values('Importance', ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance)
        plt.title(f'Feature Importance in {name} Model (Combined Stations)')
        plt.xlabel('Importance' if name != 'Linear Regression' else 'Absolute Coefficient Value')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.savefig(f'feature_importance_combined_stations_{name.lower().replace(" ", "_")}.png')
        plt.close()

        return feature_importance.head(3)
    except Exception as e:
        print(f"Error plotting feature importance for {name}: {str(e)}")
        return None

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred, name):
    try:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel('Actual NO2 Ground Concentration (Scaled)')
        plt.ylabel('Predicted NO2 Ground Concentration (Scaled)')
        plt.title(f'Actual vs Predicted NO2 Ground Concentrations ({name}, Combined Stations)')
        plt.tight_layout()
        plt.savefig(f'actual_vs_predicted_combined_stations_{name.lower().replace(" ", "_")}.png')
        plt.close()
    except Exception as e:
        print(f"Error plotting actual vs predicted for {name}: {str(e)}")

# Main execution
def main():
    file_path = 'prepared_air_quality_data.csv'

    # Load and preprocess data
    print("Loading and preprocessing data...")
    result = load_and_preprocess_data(file_path, exclude_stations=['TV_Center', 'Tv_Center'])
    if result[0] is None:
        print("Failed to preprocess data. Exiting.")
        return

    X, y, df, features, preprocessor, target_scaler = result
    print(f"Combined data from stations: {df['Station'].unique().tolist()}")

    # Train models
    print("Training models...")
    result = train_models(X, y, preprocessor, df)
    if result[0] is None:
        print("Failed to train models. Exiting.")
        return

    X_train, X_test, y_train, y_test, results = result

    # Process each model
    numeric_features = [f for f in features if f not in ['Station', 'Cluster']]
    categorical_features = [f for f in ['Station', 'Cluster'] if f in features]

    for name, res in results.items():
        print(f"\nResults for {name} (Combined Stations):")
        if res['best_params']:
            print(f"Best Parameters: {res['best_params']}")
        print(f"Mean Squared Error: {res['mse']:.4f}")
        print(f"R-squared Score: {res['r2']:.4f}")
        print(f"Cross-validated R-squared Scores: {res['cv_scores']}")
        print(f"Mean CV R-squared: {res['cv_scores'].mean():.4f}")

        # Feature selection
        selected_features, X_selected = select_important_features(
            res['model'], name, features, numeric_features, categorical_features, X, threshold=0.1
        )
        print(f"Selected Features: {selected_features}")

        # Plot feature importance
        top_features = plot_feature_importance(res['model'], name, features, numeric_features, categorical_features)
        if top_features is not None:
            print(f"Top 3 Features:\n{top_features}")

        # Plot actual vs predicted
        plot_actual_vs_predicted(y_test, res['y_pred'], name)

        # Per-station performance
        print(f"\nPer-Station Performance for {name}:")
        for station, metrics in res['station_metrics'].items():
            print(f"Station {station}: MSE = {metrics['MSE']:.4f}, R2 = {metrics['R2']:.4f}")

        print(f"Plots saved as 'feature_importance_combined_stations_{name.lower().replace(' ', '_')}.png' and 'actual_vs_predicted_combined_stations_{name.lower().replace(' ', '_')}.png'")

if __name__ == "__main__":
    main()

Loading and preprocessing data...
Correlation between NO2_Satellite and NO2_Ground in combined data (after log-transform): -0.0063
Combined data from stations: ['agrabad', 'barisal', 'cumilla', 'darussalam', 'doe', 'gazipur', 'khulna', 'mymensingh', 'ng', 'ns', 'rangpur', 'savar', 'sylhet', 'rajshahi', 'brac']
Training models...

Results for Random Forest (Combined Stations):
Best Parameters: {'rf__max_depth': 10, 'rf__max_features': 0.5, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}
Mean Squared Error: 0.0093
R-squared Score: 0.5706
Cross-validated R-squared Scores: [0.56589439 0.54841128 0.54188049 0.55691856 0.55873139]
Mean CV R-squared: 0.5544
Error in feature selection for Random Forest: 'random_forest'
Selected Features: ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Temperature', 'Wind_Speed', 'Wind_Direction', 'Month', 'DayOfWeek', 'Temp_RH_Interaction', 'Wind_Speed_Direction', 'Satellite_Solar_Interaction', 'Satellite_Month_Intera

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the combined data
def load_and_preprocess_data(file_path, exclude_stations=['tv_center'], min_samples=10):
    try:
        df = pd.read_csv(file_path)

        # Standardize station names to lowercase
        df['Station'] = df['Station'].str.lower()
        exclude_stations = [s.lower() for s in exclude_stations]

        # Exclude specified stations
        df = df[~df['Station'].isin(exclude_stations)]

        if df.empty:
            print(f"No data available after excluding stations {exclude_stations}.")
            return None, None, None, None, None, None

        # Check for constant or insufficient target data
        if df['NO2_Ground'].nunique() <= 1:
            print(f"Target NO2_Ground is constant in combined data. Skipping modeling.")
            return None, None, None, None, None, None
        if len(df) < min_samples:
            print(f"Insufficient samples ({len(df)}) in combined data. Minimum required: {min_samples}.")
            return None, None, None, None, None, None

        # Converting Date to datetime
        df['Date'] = pd.to_datetime(df['Date'])

        # Select numeric columns for averaging
        numeric_columns = ['NO2_Ground', 'NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                          'Temperature', 'Wind_Speed', 'Wind_Direction']
        df_numeric = df[['Date', 'Station'] + [col for col in numeric_columns if col in df.columns]]

        # Handling duplicates by averaging numeric columns per Date and Station
        df_numeric = df_numeric.groupby(['Date', 'Station']).mean().reset_index()

        # Station-specific scaling for NO2_Satellite
        if 'NO2_Satellite' in df_numeric.columns:
            for station in df_numeric['Station'].unique():
                mask = df_numeric['Station'] == station
                if mask.sum() > 0:
                    scaler = StandardScaler()
                    df_numeric.loc[mask, 'NO2_Satellite'] = scaler.fit_transform(
                        df_numeric.loc[mask, ['NO2_Satellite']]
                    )

        # Handling missing values
        for col in numeric_columns:
            if col in df_numeric.columns:
                df_numeric[col] = df_numeric[col].fillna(df_numeric[col].mean())
        if 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Direction'] = df_numeric['Wind_Direction'].fillna(df_numeric['Wind_Direction'].median())

        # Log-transform skewed features
        for col in ['Relative_Humidity', 'Solar_Radiation', 'Wind_Speed']:
            if col in df_numeric.columns:
                df_numeric[col] = np.log1p(df_numeric[col].clip(lower=0))  # Avoid log(0)

        # Scale NO2_Ground
        target_scaler = StandardScaler()
        df_numeric['NO2_Ground'] = target_scaler.fit_transform(df_numeric[['NO2_Ground']])

        # Check for constant features and remove them
        for col in df_numeric.columns:
            if col not in ['Date', 'Station', 'NO2_Ground'] and df_numeric[col].nunique() <= 1:
                print(f"Feature {col} is constant in combined data. Dropping.")
                df_numeric = df_numeric.drop(columns=[col])

        # Stricter outlier removal (1.5 * IQR)
        def remove_outliers(df, column):
            if column in df.columns and not df[column].empty:
                Q1 = df[column].quantile(0.25)
                Q3 = df[column].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
            return df

        df_numeric = remove_outliers(df_numeric, 'NO2_Ground')

        # Feature engineering
        df_numeric['Month'] = df_numeric['Date'].dt.month
        df_numeric['DayOfWeek'] = df_numeric['Date'].dt.dayofweek
        if 'Temperature' in df_numeric.columns and 'Relative_Humidity' in df_numeric.columns:
            df_numeric['Temp_RH_Interaction'] = df_numeric['Temperature'] * df_numeric['Relative_Humidity']
        else:
            df_numeric['Temp_RH_Interaction'] = 0
        if 'Wind_Speed' in df_numeric.columns and 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Speed_Direction'] = df_numeric['Wind_Speed'] * np.cos(np.radians(df_numeric['Wind_Direction']))
        else:
            df_numeric['Wind_Speed_Direction'] = 0
        if 'NO2_Satellite' in df_numeric.columns and 'Solar_Radiation' in df_numeric.columns:
            df_numeric['Satellite_Solar_Interaction'] = df_numeric['NO2_Satellite'] * df_numeric['Solar_Radiation']
        else:
            df_numeric['Satellite_Solar_Interaction'] = 0
        if 'NO2_Satellite' in df_numeric.columns:
            df_numeric['Satellite_Month_Interaction'] = df_numeric['NO2_Satellite'] * df_numeric['Month']
        else:
            df_numeric['Satellite_Month_Interaction'] = 0

        # Station clustering
        cluster_features = ['NO2_Ground', 'NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Temperature']
        cluster_data = df_numeric[[f for f in cluster_features if f in df_numeric.columns]].fillna(0)
        if len(cluster_data) > 0:
            kmeans = KMeans(n_clusters=5, random_state=42)
            df_numeric['Cluster'] = kmeans.fit_predict(cluster_data)
        else:
            df_numeric['Cluster'] = 0

        # Selecting features and target
        features = ['NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Temperature',
                    'Wind_Speed', 'Wind_Direction', 'Month', 'DayOfWeek', 'Temp_RH_Interaction',
                    'Wind_Speed_Direction', 'Satellite_Solar_Interaction', 'Satellite_Month_Interaction',
                    'Station', 'Cluster']
        features = [f for f in features if f in df_numeric.columns]
        if not features or 'NO2_Ground' not in df_numeric.columns:
            print(f"Insufficient features or target in combined data.")
            return None, None, None, None, None, None

        # Correlation analysis
        if 'NO2_Satellite' in df_numeric.columns:
            correlation = df_numeric[['NO2_Satellite', 'NO2_Ground']].corr().iloc[0, 1]
            print(f"Correlation between NO2_Satellite and NO2_Ground in combined data (after transformations): {correlation:.4f}")

        X = df_numeric[features]
        y = df_numeric['NO2_Ground']

        # Define preprocessing pipeline with polynomial features
        numeric_features = [f for f in features if f not in ['Station', 'Cluster']]
        categorical_features = [f for f in ['Station', 'Cluster'] if f in features]

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', Pipeline([
                    ('scaler', StandardScaler()),
                    ('poly', PolynomialFeatures(degree=2, include_bias=False))
                ]), numeric_features),
                ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
            ])

        return X, y, df_numeric, features, preprocessor, target_scaler
    except Exception as e:
        print(f"Error preprocessing combined data: {str(e)}")
        return None, None, None, None, None, None

# Training and comparing models
def train_models(X, y, preprocessor, df):
    try:
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Get station labels for test set
        test_indices = X_test.index
        stations_test = df.loc[test_indices, 'Station']

        # Define models and pipelines
        models = {
            'Random Forest': Pipeline([
                ('preprocessor', preprocessor),
                ('rf', RandomForestRegressor(random_state=42))
            ]),
            'XGBoost': Pipeline([
                ('preprocessor', preprocessor),
                ('xgb', XGBRegressor(random_state=42))
            ])
        }

        # Parameter grids
        rf_param_grid = {
            'rf__n_estimators': [100, 200, 300],
            'rf__max_depth': [5, 10, 15],
            'rf__min_samples_split': [2, 5],
            'rf__min_samples_leaf': [1, 2],
            'rf__max_features': ['sqrt', 0.5]
        }
        xgb_param_grid = {
            'xgb__n_estimators': [100, 200, 300],
            'xgb__max_depth': [3, 5, 7],
            'xgb__learning_rate': [0.01, 0.1],
            'xgb__subsample': [0.8, 1.0]
        }

        # Grid search for Random Forest and XGBoost
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        results = {}
        for name, pipeline in models.items():
            param_grid = rf_param_grid if name == 'Random Forest' else xgb_param_grid
            grid_search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1)
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_

            # Predictions and metrics
            y_pred = best_model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            cv_scores = cross_val_score(best_model, X, y, cv=kfold, scoring='r2')

            # Per-station performance
            station_metrics = {}
            for station in stations_test.unique():
                mask = stations_test == station
                if mask.sum() > 0:
                    station_mse = mean_squared_error(y_test[mask], y_pred[mask])
                    station_r2 = r2_score(y_test[mask], y_pred[mask])
                    station_metrics[station] = {'MSE': station_mse, 'R2': station_r2}

            results[name] = {
                'model': best_model,
                'y_pred': y_pred,
                'mse': mse,
                'r2': r2,
                'cv_scores': cv_scores,
                'best_params': best_params,
                'station_metrics': station_metrics
            }

        return X_train, X_test, y_train, y_test, results
    except Exception as e:
        print(f"Error training models: {str(e)}")
        return None, None, None, None, None

# Feature selection based on importance
def select_important_features(model, name, features, numeric_features, categorical_features, X, threshold=0.05):
    try:
        if name == 'Random Forest':
            importance = model.named_steps['rf'].feature_importances_
        else:  # XGBoost
            importance = model.named_steps['xgb'].feature_importances_

        # Get feature names after preprocessing
        preprocessor = model.named_steps['preprocessor']
        num_features = []
        poly = preprocessor.named_transformers_['num'].named_steps['poly']
        poly_features = poly.get_feature_names_out(numeric_features)
        if categorical_features:
            cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        else:
            cat_feature_names = []
        all_features = list(poly_features) + cat_feature_names

        feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importance})
        selected_features = feature_importance[feature_importance['Importance'] >= threshold]['Feature'].tolist()

        if not selected_features:
            return features, X

        # Map back to original feature names
        selected_orig_features = []
        for f in selected_features:
            if f in poly_features:
                # Extract original numeric features from polynomial terms
                for orig_f in numeric_features:
                    if orig_f in f and orig_f not in selected_orig_features:
                        selected_orig_features.append(orig_f)
            elif f in cat_feature_names:
                for cat_feature in categorical_features:
                    if f.startswith(cat_feature) and cat_feature not in selected_orig_features:
                        selected_orig_features.append(cat_feature)

        # Select corresponding columns in X
        selected_cols = [f for f in selected_orig_features if f in X.columns]

        return selected_cols, X[selected_cols]
    except Exception as e:
        print(f"Error in feature selection for {name}: {str(e)}")
        return features, X

# Visualizing feature importance
def plot_feature_importance(model, name, features, numeric_features, categorical_features):
    try:
        if name == 'Random Forest':
            importance = model.named_steps['rf'].feature_importances_
        else:  # XGBoost
            importance = model.named_steps['xgb'].feature_importances_

        # Get feature names after preprocessing
        preprocessor = model.named_steps['preprocessor']
        num_features = []
        poly = preprocessor.named_transformers_['num'].named_steps['poly']
        poly_features = poly.get_feature_names_out(numeric_features)
        if categorical_features:
            cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        else:
            cat_feature_names = []
        all_features = list(poly_features) + cat_feature_names

        feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importance})
        feature_importance = feature_importance.sort_values('Importance', ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance)
        plt.title(f'Feature Importance in {name} Model (Combined Stations)')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.savefig(f'feature_importance_combined_stations_{name.lower().replace(" ", "_")}.png')
        plt.close()

        return feature_importance.head(3)
    except Exception as e:
        print(f"Error plotting feature importance for {name}: {str(e)}")
        return None

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred, name):
    try:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel('Actual NO2 Ground Concentration (Scaled)')
        plt.ylabel('Predicted NO2 Ground Concentration (Scaled)')
        plt.title(f'Actual vs Predicted NO2 Ground Concentrations ({name}, Combined Stations)')
        plt.tight_layout()
        plt.savefig(f'actual_vs_predicted_combined_stations_{name.lower().replace(" ", "_")}.png')
        plt.close()
    except Exception as e:
        print(f"Error plotting actual vs predicted for {name}: {str(e)}")

# Main execution
def main():
    file_path = 'prepared_air_quality_data.csv'

    # Load and preprocess data
    print("Loading and preprocessing data...")
    result = load_and_preprocess_data(file_path, exclude_stations=['TV_Center', 'Tv_Center'])
    if result[0] is None:
        print("Failed to preprocess data. Exiting.")
        return

    X, y, df, features, preprocessor, target_scaler = result
    print(f"Combined data from stations: {df['Station'].unique().tolist()}")

    # Train models
    print("Training models...")
    result = train_models(X, y, preprocessor, df)
    if result[0] is None:
        print("Failed to train models. Exiting.")
        return

    X_train, X_test, y_train, y_test, results = result

    # Process each model
    numeric_features = [f for f in features if f not in ['Station', 'Cluster']]
    categorical_features = [f for f in ['Station', 'Cluster'] if f in features]

    for name, res in results.items():
        print(f"\nResults for {name} (Combined Stations):")
        print(f"Best Parameters: {res['best_params']}")
        print(f"Mean Squared Error: {res['mse']:.4f}")
        print(f"R-squared Score: {res['r2']:.4f}")
        print(f"Cross-validated R-squared Scores: {res['cv_scores']}")
        print(f"Mean CV R-squared: {res['cv_scores'].mean():.4f}")

        # Feature selection
        selected_features, X_selected = select_important_features(
            res['model'], name, features, numeric_features, categorical_features, X, threshold=0.05
        )
        print(f"Selected Features: {selected_features}")

        # Plot feature importance
        top_features = plot_feature_importance(res['model'], name, features, numeric_features, categorical_features)
        if top_features is not None:
            print(f"Top 3 Features:\n{top_features}")

        # Plot actual vs predicted
        plot_actual_vs_predicted(y_test, res['y_pred'], name)

        # Per-station performance
        print(f"\nPer-Station Performance for {name}:")
        for station, metrics in res['station_metrics'].items():
            print(f"Station {station}: MSE = {metrics['MSE']:.4f}, R2 = {metrics['R2']:.4f}")

        print(f"Plots saved as 'feature_importance_combined_stations_{name.lower().replace(' ', '_')}.png' and 'actual_vs_predicted_combined_stations_{name.lower().replace(' ', '_')}.png'")

if __name__ == "__main__":
    main()

Loading and preprocessing data...
Correlation between NO2_Satellite and NO2_Ground in combined data (after transformations): 0.0798
Combined data from stations: ['agrabad', 'barisal', 'cumilla', 'darussalam', 'doe', 'gazipur', 'khulna', 'mymensingh', 'ng', 'ns', 'rangpur', 'savar', 'sylhet', 'rajshahi', 'brac']
Training models...

Results for Random Forest (Combined Stations):
Best Parameters: {'rf__max_depth': 15, 'rf__max_features': 0.5, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 300}
Mean Squared Error: 0.0077
R-squared Score: 0.6435
Cross-validated R-squared Scores: [0.64190297 0.64809426 0.62493411 0.64056884 0.65093233]
Mean CV R-squared: 0.6413
Selected Features: ['Solar_Radiation', 'Station']
Top 3 Features:
              Feature  Importance
90    Station_agrabad    0.103006
35  Solar_Radiation^2    0.078885
97     Station_khulna    0.040329

Per-Station Performance for Random Forest:
Station doe: MSE = 0.0148, R2 = 0.2085
Station rajshahi: MSE =

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the combined data
def load_and_preprocess_data(file_path, exclude_stations=['tv_center'], min_samples=10):
    try:
        df = pd.read_csv(file_path)

        # Standardize station names to lowercase
        df['Station'] = df['Station'].str.lower()
        exclude_stations = [s.lower() for s in exclude_stations]

        # Exclude specified stations
        df = df[~df['Station'].isin(exclude_stations)]

        if df.empty:
            print(f"No data available after excluding stations {exclude_stations}.")
            return None, None, None, None, None, None

        # Check for constant or insufficient target data
        if df['NO2_Ground'].nunique() <= 1:
            print(f"Target NO2_Ground is constant in combined data. Skipping modeling.")
            return None, None, None, None, None, None
        if len(df) < min_samples:
            print(f"Insufficient samples ({len(df)}) in combined data. Minimum required: {min_samples}.")
            return None, None, None, None, None, None

        # Converting Date to datetime
        df['Date'] = pd.to_datetime(df['Date'])

        # Select numeric columns for averaging
        numeric_columns = ['NO2_Ground', 'NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                          'Temperature', 'Wind_Speed', 'Wind_Direction']
        df_numeric = df[['Date', 'Station'] + [col for col in numeric_columns if col in df.columns]]

        # Handling duplicates by averaging numeric columns per Date and Station
        df_numeric = df_numeric.groupby(['Date', 'Station']).mean().reset_index()

        # Log-transform NO2_Satellite with offset
        if 'NO2_Satellite' in df_numeric.columns:
            df_numeric['NO2_Satellite'] = np.log1p(df_numeric['NO2_Satellite'].clip(lower=0) + 0.1)

        # Handling missing values
        for col in numeric_columns:
            if col in df_numeric.columns:
                df_numeric[col] = df_numeric[col].fillna(df_numeric[col].mean())
        if 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Direction'] = df_numeric['Wind_Direction'].fillna(df_numeric['Wind_Direction'].median())

        # Log-transform skewed features
        for col in ['Relative_Humidity', 'Solar_Radiation', 'Wind_Speed']:
            if col in df_numeric.columns:
                df_numeric[col] = np.log1p(df_numeric[col].clip(lower=0))  # Avoid log(0)

        # Scale NO2_Ground
        target_scaler = StandardScaler()
        df_numeric['NO2_Ground'] = target_scaler.fit_transform(df_numeric[['NO2_Ground']])

        # Station-specific outlier removal for low-performing stations (less aggressive)
        def remove_outliers(df, column, stations, iqr_factor=2.0):
            for station in stations:
                mask = df['Station'] == station
                if mask.sum() > 0:
                    Q1 = df.loc[mask, column].quantile(0.25)
                    Q3 = df.loc[mask, column].quantile(0.75)
                    IQR = Q3 - Q1
                    lower_bound = Q1 - iqr_factor * IQR
                    upper_bound = Q3 + iqr_factor * IQR
                    df = df[~mask | ((df[column] >= lower_bound) & (df[column] <= upper_bound))]
            return df

        df_numeric = remove_outliers(df_numeric, 'NO2_Ground', ['doe', 'rajshahi', 'rangpur'])

        # Check for constant features and remove them
        for col in df_numeric.columns:
            if col not in ['Date', 'Station', 'NO2_Ground'] and df_numeric[col].nunique() <= 1:
                print(f"Feature {col} is constant in combined data. Dropping.")
                df_numeric = df_numeric.drop(columns=[col])

        # Feature engineering
        df_numeric['Month'] = df_numeric['Date'].dt.month
        df_numeric['DayOfWeek'] = df_numeric['Date'].dt.dayofweek
        if 'Temperature' in df_numeric.columns and 'Relative_Humidity' in df_numeric.columns:
            df_numeric['Temp_RH_Interaction'] = df_numeric['Temperature'] * df_numeric['Relative_Humidity']
        else:
            df_numeric['Temp_RH_Interaction'] = 0
        if 'Wind_Speed' in df_numeric.columns and 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Speed_Direction'] = df_numeric['Wind_Speed'] * np.cos(np.radians(df_numeric['Wind_Direction']))
        else:
            df_numeric['Wind_Speed_Direction'] = 0
        if 'NO2_Satellite' in df_numeric.columns and 'Solar_Radiation' in df_numeric.columns:
            df_numeric['Satellite_Solar_Interaction'] = df_numeric['NO2_Satellite'] * df_numeric['Solar_Radiation']
        else:
            df_numeric['Satellite_Solar_Interaction'] = 0
        if 'NO2_Satellite' in df_numeric.columns:
            df_numeric['Satellite_Month_Interaction'] = df_numeric['NO2_Satellite'] * df_numeric['Month']
        else:
            df_numeric['Satellite_Month_Interaction'] = 0

        # Add lagged NO2_Satellite
        if 'NO2_Satellite' in df_numeric.columns:
            df_numeric['NO2_Satellite_Lag1'] = df_numeric.groupby('Station')['NO2_Satellite'].shift(1)
            df_numeric['NO2_Satellite_Lag1'] = df_numeric['NO2_Satellite_Lag1'].fillna(df_numeric['NO2_Satellite'].mean())

        # Station clustering
        cluster_features = ['NO2_Ground', 'NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Temperature']
        cluster_data = df_numeric[[f for f in cluster_features if f in df_numeric.columns]].fillna(0)
        if len(cluster_data) > 0:
            kmeans = KMeans(n_clusters=4, random_state=42)
            df_numeric['Cluster'] = kmeans.fit_predict(cluster_data)
        else:
            df_numeric['Cluster'] = 0

        # Add NO2_Satellite × Cluster interaction
        if 'NO2_Satellite' in df_numeric.columns:
            for cluster in df_numeric['Cluster'].unique():
                df_numeric[f'Satellite_Cluster_{cluster}'] = df_numeric['NO2_Satellite'] * (df_numeric['Cluster'] == cluster).astype(int)

        # Selecting features and target
        features = ['NO2_Satellite', 'NO2_Satellite_Lag1', 'Relative_Humidity', 'Solar_Radiation',
                    'Temperature', 'Wind_Speed', 'Wind_Direction', 'Month', 'DayOfWeek',
                    'Temp_RH_Interaction', 'Wind_Speed_Direction', 'Satellite_Solar_Interaction',
                    'Satellite_Month_Interaction', 'Station', 'Cluster'] + \
                   [f for f in df_numeric.columns if f.startswith('Satellite_Cluster_')]
        features = [f for f in features if f in df_numeric.columns]
        if not features or 'NO2_Ground' not in df_numeric.columns:
            print(f"Insufficient features or target.")
            return None, None, None, None, None, None

        # Correlation analysis
        if 'NO2_Satellite' in df_numeric.columns:
            correlation = df_numeric[['NO2_Ground', 'NO2_Satellite']].corr().iloc[0, 1]
            print(f"Correlation between NO2_Satellite and NO2_Ground (after log transform with offset): {correlation:.4f}")

        X = df_numeric[features]
        y = df_numeric['NO2_Ground']

        # Define sample weights based on station performance
        station_weights = {
            'khulna': 1.5, 'gazipur': 1.5, 'barisal': 1.3, 'brac': 1.2, 'agrabad': 1.2,
            'doe': 0.8, 'rajshahi': 0.8, 'rangpur': 0.8
        }
        sample_weights = df_numeric['Station'].map(station_weights).fillna(1.0)

        # Define preprocessing pipeline
        numeric_features = [f for f in features if f not in ['Station', 'Cluster']]
        categorical_features = [f for f in ['Station', 'Cluster'] if f in features]

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_features),
                ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
            ])

        return X, y, df_numeric, features, preprocessor, target_scaler, sample_weights
    except Exception as e:
        print(f"Error preprocessing data: {str(e)}")
        return None, None, None, None, None, None, None

# Training and comparing models
def train_models(X, y, preprocessor, df, sample_weights):
    try:
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Get station labels and weights for test set
        test_indices = X_test.index
        stations_test = df.loc[test_indices, 'Station']
        weights_train = sample_weights.loc[X_train.index]

        # Define models and pipelines
        models = {
            'Random Forest': Pipeline([
                ('preprocessor', preprocessor),
                ('rf', RandomForestRegressor(random_state=42))
            ]),
            'XGBoost': Pipeline([
                ('preprocessor', preprocessor),
                ('xgb', XGBRegressor(random_state=42))
            ])
        }

        # Parameter grids
        rf_param_grid = {
            'rf__n_estimators': [100, 200, 300],
            'rf__max_depth': [5, 10, 12],
            'rf__min_samples_split': [2, 5],
            'rf__min_samples_leaf': [1, 2],
            'rf__max_features': ['sqrt', 0.5]
        }
        xgb_param_grid = {
            'xgb__n_estimators': [100, 200, 300],
            'xgb__max_depth': [3, 5, 7],
            'xgb__learning_rate': [0.01, 0.05, 0.1],
            'xgb__subsample': [0.8, 1.0],
            'xgb__reg_lambda': [0.1, 1.0, 10.0]
        }

        # Grid search for Random Forest and XGBoost
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        results = {}
        for name, pipeline in models.items():
            param_grid = rf_param_grid if name == 'Random Forest' else xgb_param_grid
            grid_search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1)
            grid_search.fit(X_train, y_train, **{'rf__sample_weight': weights_train} if name == 'Random Forest' else {})
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_

            # Predictions and metrics
            y_pred = best_model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            cv_scores = cross_val_score(best_model, X, y, cv=kfold, scoring='r2')

            # Per-station performance
            station_metrics = {}
            for station in stations_test.unique():
                mask = stations_test == station
                if mask.sum() > 0:
                    station_mse = mean_squared_error(y_test[mask], y_pred[mask])
                    station_r2 = r2_score(y_test[mask], y_pred[mask])
                    station_metrics[station] = {'MSE': station_mse, 'R2': station_r2}

            results[name] = {
                'model': best_model,
                'y_pred': y_pred,
                'mse': mse,
                'r2': r2,
                'cv_scores': cv_scores,
                'best_params': best_params,
                'station_metrics': station_metrics
            }

        return X_train, X_test, y_train, y_test, results
    except Exception as e:
        print(f"Error training models: {str(e)}")
        return None, None, None, None, None

# Feature selection based on importance
def select_important_features(model, name, features, numeric_features, categorical_features, X, threshold=0.03):
    try:
        if name == 'Random Forest':
            importance = model.named_steps['rf'].feature_importances_
        else:  # XGBoost
            importance = model.named_steps['xgb'].feature_importances_

        # Get feature names after preprocessing
        preprocessor = model.named_steps['preprocessor']
        num_features = numeric_features
        if categorical_features:
            cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        else:
            cat_feature_names = []
        all_features = num_features + cat_feature_names

        feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importance})
        selected_features = feature_importance[feature_importance['Importance'] >= threshold]['Feature'].tolist()

        if not selected_features:
            return features, X

        # Ensure NO2_Satellite-related features are included
        satellite_features = [f for f in all_features if 'NO2_Satellite' in f]
        if not any('NO2_Satellite' in f for f in selected_features):
            selected_features.extend(satellite_features[:2])  # Include top 2 NO2_Satellite features

        # Map back to original feature names
        selected_orig_features = []
        for f in selected_features:
            if f in num_features:
                selected_orig_features.append(f)
            else:
                for cat_feature in categorical_features:
                    if f.startswith(cat_feature) and cat_feature not in selected_orig_features:
                        selected_orig_features.append(cat_feature)

        # Select corresponding columns in X
        selected_cols = [f for f in selected_orig_features if f in X.columns]

        return selected_cols, X[selected_cols]
    except Exception as e:
        print(f"Error in feature selection for {name}: {str(e)}")
        return features, X

# Visualizing feature importance
def plot_feature_importance(model, name, features, numeric_features, categorical_features):
    try:
        if name == 'Random Forest':
            importance = model.named_steps['rf'].feature_importances_
        else:  # XGBoost
            importance = model.named_steps['xgb'].feature_importances_

        # Get feature names after preprocessing
        preprocessor = model.named_steps['preprocessor']
        num_features = numeric_features
        if categorical_features:
            cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        else:
            cat_feature_names = []
        all_features = num_features + cat_feature_names

        feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importance})
        feature_importance = feature_importance.sort_values('Importance', ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance)
        plt.title(f'Feature Importance in {name} Model (Combined Stations)')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.savefig(f'feature_importance_combined_stations_{name.lower().replace(" ", "_")}.png')
        plt.close()

        return feature_importance.head(3)
    except Exception as e:
        print(f"Error plotting feature importance for {name}: {str(e)}")
        return None

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred, name):
    try:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel('Actual NO2 Ground Concentration (Scaled)')
        plt.ylabel('Predicted NO2 Ground Concentration (Scaled)')
        plt.title(f'Actual vs Predicted NO2 Ground Concentrations ({name}, Combined Stations)')
        plt.tight_layout()
        plt.savefig(f'actual_vs_predicted_combined_stations_{name.lower().replace(" ", "_")}.png')
        plt.close()
    except Exception as e:
        print(f"Error plotting actual vs predicted for {name}: {str(e)}")

# Main execution
def main():
    file_path = 'prepared_air_quality_data.csv'

    # Load and preprocess data
    print("Loading and preprocessing data...")
    result = load_and_preprocess_data(file_path, exclude_stations=['TV_Center', 'Tv_Center'])
    if result[0] is None:
        print("Failed to preprocess data. Exiting.")
        return

    X, y, df, features, preprocessor, target_scaler, sample_weights = result
    print(f"Combined data from stations: {df['Station'].unique().tolist()}")

    # Train models
    print("Training models...")
    result = train_models(X, y, preprocessor, df, sample_weights)
    if result[0] is None:
        print("Failed to train models. Exiting.")
        return

    X_train, X_test, y_train, y_test, results = result

    # Process each model
    numeric_features = [f for f in features if f not in ['Station', 'Cluster']]
    categorical_features = [f for f in ['Station', 'Cluster'] if f in features]

    for name, res in results.items():
        print(f"\nResults for {name} (Combined Stations):")
        print(f"Best Parameters: {res['best_params']}")
        print(f"Mean Squared Error: {res['mse']:.4f}")
        print(f"R-squared Score: {res['r2']:.4f}")
        print(f"Cross-validated R-squared Scores: {res['cv_scores']}")
        print(f"Mean CV R-squared: {res['cv_scores'].mean():.4f}")

        # Feature selection
        selected_features, X_selected = select_important_features(
            res['model'], name, features, numeric_features, categorical_features, X, threshold=0.03
        )
        print(f"Selected Features: {selected_features}")

        # Plot feature importance
        top_features = plot_feature_importance(res['model'], name, features, numeric_features, categorical_features)
        if top_features is not None:
            print(f"Top 3 Features:\n{top_features}")

        # Plot actual vs predicted
        plot_actual_vs_predicted(y_test, res['y_pred'], name)

        # Per-station performance
        print(f"\nPer-Station Performance for {name}:")
        for station, metrics in res['station_metrics'].items():
            print(f"Station {station}: MSE = {metrics['MSE']:.4f}, R2 = {metrics['R2']:.4f}")

        print(f"Plots saved as 'feature_importance_combined_stations_{name.lower().replace(' ', '_')}.png' and 'actual_vs_predicted_combined_stations_{name.lower().replace(' ', '_')}.png'")

if __name__ == "__main__":
    main()

Loading and preprocessing data...
Correlation between NO2_Satellite and NO2_Ground (after log transform with offset): 0.1398
Combined data from stations: ['agrabad', 'barisal', 'brac', 'cumilla', 'darussalam', 'doe', 'gazipur', 'khulna', 'mymensingh', 'ng', 'ns', 'rangpur', 'savar', 'sylhet', 'rajshahi']
Training models...

Results for Random Forest (Combined Stations):
Best Parameters: {'rf__max_depth': 12, 'rf__max_features': 0.5, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 300}
Mean Squared Error: 0.3762
R-squared Score: 0.6172
Cross-validated R-squared Scores: [0.62266605 0.71588373 0.63722536 0.52991281 0.71701038]
Mean CV R-squared: 0.6445
Selected Features: ['NO2_Satellite', 'NO2_Satellite_Lag1', 'Relative_Humidity', 'Solar_Radiation', 'Temperature', 'Wind_Speed', 'Wind_Direction', 'Temp_RH_Interaction', 'Wind_Speed_Direction', 'Satellite_Solar_Interaction', 'Satellite_Month_Interaction', 'Satellite_Cluster_1', 'Station']
Top 3 Features:
          

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and preprocessing the combined data
def load_and_preprocess_data(file_path, exclude_stations=['tv_center', 'doe', 'ns', 'cumilla'], min_samples=10):
    try:
        df = pd.read_csv(file_path)

        # Standardize station names to lowercase
        df['Station'] = df['Station'].str.lower()
        exclude_stations = [s.lower() for s in exclude_stations]

        # Exclude specified stations
        df = df[~df['Station'].isin(exclude_stations)]

        if df.empty:
            print(f"No data available after excluding stations {exclude_stations}.")
            return None, None, None, None, None, None

        # Check for constant or insufficient target data
        if df['NO2_Ground'].nunique() <= 1:
            print(f"Target NO2_Ground is constant in combined data. Skipping modeling.")
            return None, None, None, None, None, None
        if len(df) < min_samples:
            print(f"Insufficient samples ({len(df)}) in combined data. Minimum required: {min_samples}.")
            return None, None, None, None, None, None

        # Converting Date to datetime
        df['Date'] = pd.to_datetime(df['Date'])

        # Select numeric columns for averaging
        numeric_columns = ['NO2_Ground', 'NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation',
                          'Temperature', 'Wind_Speed', 'Wind_Direction']
        df_numeric = df[['Date', 'Station'] + [col for col in numeric_columns if col in df.columns]]

        # Handling duplicates by averaging numeric columns per Date and Station
        df_numeric = df_numeric.groupby(['Date', 'Station']).mean().reset_index()

        # Square root transform NO2_Satellite
        if 'NO2_Satellite' in df_numeric.columns:
            df_numeric['NO2_Satellite'] = np.sqrt(df_numeric['NO2_Satellite'].clip(lower=0))

        # Handling missing values
        for col in numeric_columns:
            if col in df_numeric.columns:
                df_numeric[col] = df_numeric[col].fillna(df_numeric[col].mean())
        if 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Direction'] = df_numeric['Wind_Direction'].fillna(df_numeric['Wind_Direction'].median())

        # Log-transform skewed features
        for col in ['Relative_Humidity', 'Solar_Radiation', 'Wind_Speed']:
            if col in df_numeric.columns:
                df_numeric[col] = np.log1p(df_numeric[col].clip(lower=0))  # Avoid log(0)

        # Scale NO2_Ground
        target_scaler = StandardScaler()
        df_numeric['NO2_Ground'] = target_scaler.fit_transform(df_numeric[['NO2_Ground']])

        # Station-specific outlier removal for low-performing stations
        def remove_outliers(df, column, stations, iqr_factor=1.5):
            for station in stations:
                mask = df['Station'] == station
                if mask.sum() > 0:
                    Q1 = df.loc[mask, column].quantile(0.25)
                    Q3 = df.loc[mask, column].quantile(0.75)
                    IQR = Q3 - Q1
                    lower_bound = Q1 - iqr_factor * IQR
                    upper_bound = Q3 + iqr_factor * IQR
                    df = df[~mask | ((df[column] >= lower_bound) & (df[column] <= upper_bound))]
            return df

        df_numeric = remove_outliers(df_numeric, 'NO2_Ground', ['rajshahi', 'rangpur'])

        # Check for constant features and remove them
        for col in df_numeric.columns:
            if col not in ['Date', 'Station', 'NO2_Ground'] and df_numeric[col].nunique() <= 1:
                print(f"Feature {col} is constant in combined data. Dropping.")
                df_numeric = df_numeric.drop(columns=[col])

        # Feature engineering
        df_numeric['Month'] = df_numeric['Date'].dt.month
        df_numeric['DayOfWeek'] = df_numeric['Date'].dt.dayofweek
        if 'Temperature' in df_numeric.columns and 'Relative_Humidity' in df_numeric.columns:
            df_numeric['Temp_RH_Interaction'] = df_numeric['Temperature'] * df_numeric['Relative_Humidity']
        else:
            df_numeric['Temp_RH_Interaction'] = 0
        if 'Wind_Speed' in df_numeric.columns and 'Wind_Direction' in df_numeric.columns:
            df_numeric['Wind_Speed_Direction'] = df_numeric['Wind_Speed'] * np.cos(np.radians(df_numeric['Wind_Direction']))
        else:
            df_numeric['Wind_Speed_Direction'] = 0
        if 'NO2_Satellite' in df_numeric.columns and 'Solar_Radiation' in df_numeric.columns:
            df_numeric['Satellite_Solar_Interaction'] = df_numeric['NO2_Satellite'] * df_numeric['Solar_Radiation']
        else:
            df_numeric['Satellite_Solar_Interaction'] = 0
        if 'NO2_Satellite' in df_numeric.columns:
            df_numeric['Satellite_Month_Interaction'] = df_numeric['NO2_Satellite'] * df_numeric['Month']
        else:
            df_numeric['Satellite_Month_Interaction'] = 0

        # Add lagged NO2_Satellite
        if 'NO2_Satellite' in df_numeric.columns:
            df_numeric['NO2_Satellite_Lag1'] = df_numeric.groupby('Station')['NO2_Satellite'].shift(1)
            df_numeric['NO2_Satellite_Lag1'] = df_numeric['NO2_Satellite_Lag1'].fillna(df_numeric['NO2_Satellite'].mean())

        # Add NO2_Satellite × Station interactions
        if 'NO2_Satellite' in df_numeric.columns:
            for station in df_numeric['Station'].unique():
                df_numeric[f'Satellite_Station_{station}'] = df_numeric['NO2_Satellite'] * (df_numeric['Station'] == station).astype(int)

        # Station clustering
        cluster_features = ['NO2_Ground', 'NO2_Satellite', 'Relative_Humidity', 'Solar_Radiation', 'Temperature']
        cluster_data = df_numeric[[f for f in cluster_features if f in df_numeric.columns]].fillna(0)
        if len(cluster_data) > 0:
            kmeans = KMeans(n_clusters=3, random_state=42)
            df_numeric['Cluster'] = kmeans.fit_predict(cluster_data)
        else:
            df_numeric['Cluster'] = 0

        # Add NO2_Satellite × Cluster interaction
        if 'NO2_Satellite' in df_numeric.columns:
            for cluster in df_numeric['Cluster'].unique():
                df_numeric[f'Satellite_Cluster_{cluster}'] = df_numeric['NO2_Satellite'] * (df_numeric['Cluster'] == cluster).astype(int)

        # Selecting features and target
        features = ['NO2_Satellite', 'NO2_Satellite_Lag1', 'Relative_Humidity', 'Solar_Radiation',
                    'Temperature', 'Wind_Speed', 'Wind_Direction', 'Month', 'DayOfWeek',
                    'Temp_RH_Interaction', 'Wind_Speed_Direction', 'Satellite_Solar_Interaction',
                    'Satellite_Month_Interaction', 'Station', 'Cluster'] + \
                   [f for f in df_numeric.columns if f.startswith('Satellite_Station_') or f.startswith('Satellite_Cluster_')]
        features = [f for f in features if f in df_numeric.columns]
        if not features or 'NO2_Ground' not in df_numeric.columns:
            print(f"Insufficient features or target.")
            return None, None, None, None, None, None

        # Correlation analysis
        if 'NO2_Satellite' in df_numeric.columns:
            correlation = df_numeric[['NO2_Ground', 'NO2_Satellite']].corr().iloc[0, 1]
            print(f"Correlation between NO2_Satellite and NO2_Ground (after sqrt transform): {correlation:.4f}")

        X = df_numeric[features]
        y = df_numeric['NO2_Ground']

        # Define preprocessing pipeline
        numeric_features = [f for f in features if f not in ['Station', 'Cluster']]
        categorical_features = [f for f in ['Station', 'Cluster'] if f in features]

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_features),
                ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
            ])

        return X, y, df_numeric, features, preprocessor, target_scaler
    except Exception as e:
        print(f"Error preprocessing data: {str(e)}")
        return None, None, None, None, None, None

# Training and comparing models
def train_models(X, y, preprocessor, df):
    try:
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Get station labels for test set
        test_indices = X_test.index
        stations_test = df.loc[test_indices, 'Station']

        # Inverse transform y for evaluation
        target_scaler = StandardScaler()
        target_scaler.fit(y.values.reshape(-1, 1))
        y_train_unscaled = target_scaler.inverse_transform(y_train.values.reshape(-1, 1)).ravel()
        y_test_unscaled = target_scaler.inverse_transform(y_test.values.reshape(-1, 1)).ravel()

        # Define models and pipelines
        models = {
            'Random Forest': Pipeline([
                ('preprocessor', preprocessor),
                ('rf', RandomForestRegressor(random_state=42))
            ]),
            'XGBoost': Pipeline([
                ('preprocessor', preprocessor),
                ('xgb', XGBRegressor(random_state=42))
            ])
        }

        # Parameter grids
        rf_param_grid = {
            'rf__n_estimators': [100, 200, 300],
            'rf__max_depth': [5, 10, 12],
            'rf__min_samples_split': [2, 5],
            'rf__min_samples_leaf': [1, 2],
            'rf__max_features': ['sqrt', 0.5]
        }
        xgb_param_grid = {
            'xgb__n_estimators': [100, 200, 300],
            'xgb__max_depth': [3, 5],
            'xgb__learning_rate': [0.01, 0.05, 0.1],
            'xgb__subsample': [0.8, 1.0],
            'xgb__reg_lambda': [1.0, 10.0, 50.0]
        }

        # Grid search for Random Forest and XGBoost
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        results = {}
        for name, pipeline in models.items():
            param_grid = rf_param_grid if name == 'Random Forest' else xgb_param_grid
            grid_search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1)
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_

            # Predictions
            y_pred = best_model.predict(X_test)
            y_pred_unscaled = target_scaler.inverse_transform(y_pred.reshape(-1, 1)).ravel()

            # Metrics on unscaled data
            mse = mean_squared_error(y_test_unscaled, y_pred_unscaled)
            r2 = r2_score(y_test_unscaled, y_pred_unscaled)
            cv_scores = cross_val_score(best_model, X, y, cv=kfold, scoring='r2')

            # Per-station performance
            station_metrics = {}
            for station in stations_test.unique():
                mask = stations_test == station
                if mask.sum() > 0:
                    station_mse = mean_squared_error(y_test_unscaled[mask], y_pred_unscaled[mask])
                    station_r2 = r2_score(y_test_unscaled[mask], y_pred_unscaled[mask])
                    station_metrics[station] = {'MSE': station_mse, 'R2': station_r2}

            results[name] = {
                'model': best_model,
                'y_pred': y_pred_unscaled,
                'mse': mse,
                'r2': r2,
                'cv_scores': cv_scores,
                'best_params': best_params,
                'station_metrics': station_metrics
            }

        return X_train, X_test, y_train, y_test_unscaled, results
    except Exception as e:
        print(f"Error training models: {str(e)}")
        return None, None, None, None, None

# Feature selection based on importance
def select_important_features(model, name, features, numeric_features, categorical_features, X, threshold=0.02):
    try:
        if name == 'Random Forest':
            importance = model.named_steps['rf'].feature_importances_
        else:  # XGBoost
            importance = model.named_steps['xgb'].feature_importances_

        # Get feature names after preprocessing
        preprocessor = model.named_steps['preprocessor']
        num_features = numeric_features
        if categorical_features:
            cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        else:
            cat_feature_names = []
        all_features = num_features + cat_feature_names

        feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importance})
        selected_features = feature_importance[feature_importance['Importance'] >= threshold]['Feature'].tolist()

        if not selected_features:
            return features, X

        # Ensure NO2_Satellite-related features are included
        satellite_features = [f for f in all_features if 'NO2_Satellite' in f]
        if not any('NO2_Satellite' in f for f in selected_features):
            selected_features.extend(satellite_features[:3])  # Include top 3 NO2_Satellite features

        # Map back to original feature names
        selected_orig_features = []
        for f in selected_features:
            if f in num_features:
                selected_orig_features.append(f)
            else:
                for cat_feature in categorical_features:
                    if f.startswith(cat_feature) and cat_feature not in selected_orig_features:
                        selected_orig_features.append(cat_feature)

        # Select corresponding columns in X
        selected_cols = [f for f in selected_orig_features if f in X.columns]

        return selected_cols, X[selected_cols]
    except Exception as e:
        print(f"Error in feature selection for {name}: {str(e)}")
        return features, X

# Visualizing feature importance
def plot_feature_importance(model, name, features, numeric_features, categorical_features):
    try:
        if name == 'Random Forest':
            importance = model.named_steps['rf'].feature_importances_
        else:  # XGBoost
            importance = model.named_steps['xgb'].feature_importances_

        # Get feature names after preprocessing
        preprocessor = model.named_steps['preprocessor']
        num_features = numeric_features
        if categorical_features:
            cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        else:
            cat_feature_names = []
        all_features = num_features + cat_feature_names

        feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importance})
        feature_importance = feature_importance.sort_values('Importance', ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance)
        plt.title(f'Feature Importance in {name} Model (Combined Stations)')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.savefig(f'feature_importance_combined_stations_{name.lower().replace(" ", "_")}.png')
        plt.close()

        return feature_importance.head(3)
    except Exception as e:
        print(f"Error plotting feature importance for {name}: {str(e)}")
        return None

# Visualizing actual vs predicted values
def plot_actual_vs_predicted(y_test, y_pred, name):
    try:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel('Actual NO2 Ground Concentration')
        plt.ylabel('Predicted NO2 Ground Concentration')
        plt.title(f'Actual vs Predicted NO2 Ground Concentrations ({name}, Combined Stations)')
        plt.tight_layout()
        plt.savefig(f'actual_vs_predicted_combined_stations_{name.lower().replace(" ", "_")}.png')
        plt.close()
    except Exception as e:
        print(f"Error plotting actual vs predicted for {name}: {str(e)}")

# Main execution
def main():
    file_path = 'prepared_air_quality_data.csv'

    # Load and preprocess data
    print("Loading and preprocessing data...")
    result = load_and_preprocess_data(file_path)
    if result[0] is None:
        print("Failed to preprocess data. Exiting.")
        return

    X, y, df, features, preprocessor, target_scaler = result
    print(f"Combined data from stations: {df['Station'].unique().tolist()}")

    # Train models
    print("Training models...")
    result = train_models(X, y, preprocessor, df)
    if result[0] is None:
        print("Failed to train models. Exiting.")
        return

    X_train, X_test, y_train, y_test, results = result

    # Process each model
    numeric_features = [f for f in features if f not in ['Station', 'Cluster']]
    categorical_features = [f for f in ['Station', 'Cluster'] if f in features]

    for name, res in results.items():
        print(f"\nResults for {name} (Combined Stations):")
        print(f"Best Parameters: {res['best_params']}")
        print(f"Mean Squared Error: {res['mse']:.4f}")
        print(f"R-squared Score: {res['r2']:.4f}")
        print(f"Cross-validated R-squared Scores: {res['cv_scores']}")
        print(f"Mean CV R-squared: {res['cv_scores'].mean():.4f}")

        # Feature selection
        selected_features, X_selected = select_important_features(
            res['model'], name, features, numeric_features, categorical_features, X, threshold=0.02
        )
        print(f"Selected Features: {selected_features}")

        # Plot feature importance
        top_features = plot_feature_importance(res['model'], name, features, numeric_features, categorical_features)
        if top_features is not None:
            print(f"Top 3 Features:\n{top_features}")

        # Plot actual vs predicted
        plot_actual_vs_predicted(y_test, res['y_pred'], name)

        # Per-station performance
        print(f"\nPer-Station Performance for {name}:")
        for station, metrics in res['station_metrics'].items():
            print(f"Station {station}: MSE = {metrics['MSE']:.4f}, R2 = {metrics['R2']:.4f}")

        print(f"Plots saved as 'feature_importance_combined_stations_{name.lower().replace(' ', '_')}.png' and 'actual_vs_predicted_combined_stations_{name.lower().replace(' ', '_')}.png'")

if __name__ == "__main__":
    main()

Loading and preprocessing data...
Correlation between NO2_Satellite and NO2_Ground (after sqrt transform): 0.2235
Combined data from stations: ['agrabad', 'barisal', 'brac', 'darussalam', 'gazipur', 'khulna', 'mymensingh', 'ng', 'rangpur', 'savar', 'sylhet', 'rajshahi']
Training models...

Results for Random Forest (Combined Stations):
Best Parameters: {'rf__max_depth': 12, 'rf__max_features': 0.5, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}
Mean Squared Error: 0.2024
R-squared Score: 0.7906
Cross-validated R-squared Scores: [0.79452572 0.52283907 0.80130835 0.75305326 0.72426896]
Mean CV R-squared: 0.7192
Selected Features: ['NO2_Satellite', 'NO2_Satellite_Lag1', 'Relative_Humidity', 'Solar_Radiation', 'Temperature', 'Wind_Speed', 'Wind_Direction', 'Month', 'Temp_RH_Interaction', 'Wind_Speed_Direction', 'Satellite_Solar_Interaction', 'Satellite_Month_Interaction', 'Satellite_Station_brac', 'Satellite_Station_mymensingh', 'Satellite_Cluster_2', 'Stat